<img align="right" width="200" height="200" src="https://static.wikia.nocookie.net/lego/images/2/23/PPG_logo.png"> 

# <span style="color:#EF7C8E">Noon Perfume EDA - Capstone Project - Data Cleaning </span>
 


## <span style="color:#55BB99"> Group: Power Puff Girls</span>
### <span style="color:#726060"> Members: Wejdan Al-Ahmadi - Sara Aldubaie</span>
<hr style="border:2px solid gray"> </hr>

In [1]:
#!pip install wordcloud

In [2]:
# Import needed libraries 
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
# Read the dataset file and view it 
notes = pd.read_csv("./data/noon_parfum_notes.csv", low_memory=False)
print(f"Rows: {notes.shape[0]}\nColumns: {notes.shape[1]}")
# Using with to print all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(notes.head(2))

Rows: 10452
Columns: 102


,name,Base Note,Colour Name,Country of Origin,Department,Dispenser Type,Grade,Heart/Middle Note,Model Name,link,Fragrance Category,Scents/Notes,Size,Top Note,Fragrance Family,Long Lasting,Model Number,Notes,Scents,Set Includes,Skin Type,Product Line,Aromatherapy Type,Fragrance Notes,Fragrance Type,Ingredients,Fragnance Family,Fragrance Note,Formulation,Occasion,Perfumer,Composition,Height,Width,Depth,Includes,Scent Type,Unnamed: 37,Volume,Top notes,Middle / Heart Notes,Base notes,Perfume Scent,Anti-perspirant,Non-toxic,Olfactive Family,Fragrance Note Family,Fragrance Segment,Irritation Free,Packaging Type,Paraben Free,Scent,Long-lasting,Introduced Year,Product Dimensions,Shipping Weight,scents_notes,Package Type,Application Area,Anti-Perspirant,Launched,What's In The Box,Dimensions,Weight,Material,Product Height,Product Width/Depth,Scent Notes,Package weight in KGs,Package height,Package Width,Long-Lasting,Key Notes,Scent Family,Dimension,Note,Form,Lanched,Scent Note,Longlasting,Package thickness,Notes Contains,Recommended Usage,Skin type,Country Of Origin,Cruelty Free,Paraben/Sulphate Free,Suitable for the season,Package weight,Cruelty-free,Product Dimension,Colour Family,Container Type,Fragrance Classification,Notes Available,Tope Note,Middle Note,Scent/Notes,Sulfate Free,Suitable For,Scent Life,Fragrance
0,ELIE SAAB Le Parfum Royal EDP 90ml,"Indonesian Patchouli Essence, Indian Sandalwoo...",Gold,France,Women,Pump,New,"Turkish Rose Absolute, Bulgarian Rose Absolute...",Le Parfum Royal,/uae-en/le-parfum-royal-edp-90ml/N24430947A/p?...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LALIQUE Le Parfum EDP 100ml,"Patchouli, Sandalwood, Tonka Beans, Vanilla",NaN,NaN,Women,Splash,NaN,"Heliotrope, Jasmine",NaN,/uae-en/le-parfum-edp-100ml/N11201336A/p?o=aa9...,Oriental,Oriental,100 ml,"Bay Leaf, Bergamot, Pink Pepper",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Read the dataset file and view it 
links = pd.read_csv("./data/noon_parfum_links.csv")
print(f"Rows: {links.shape[0]}\nColumns: {links.shape[1]}")
# Using with to print all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(links.head(2))

Rows: 10500
Columns: 4


,name,currency,price,link
0,ELIE SAAB Le Parfum Royal EDP 90ml,AED,196.95,/uae-en/le-parfum-royal-edp-90ml/N24430947A/p?...
1,LALIQUE Le Parfum EDP 100ml,AED,103.70,/uae-en/le-parfum-edp-100ml/N11201336A/p?o=aa9...


## <span style="color:#55BB99"> 1- General Cleanup</span>
In this section of the notebook we do a general cleanup for the whole dataset in preparation for the EDA and the model. 

In [5]:
# Merge dataset based on the 'links' column 
noon = pd.merge(notes, links, on="link")
noon.drop_duplicates(keep=False,inplace=True)

In [6]:
# Rename the column 
noon = noon.rename(columns={'name_x': 'name'})

# Select only needed columns 
noon = noon[['name', 'Base Note', 'Department', 'Heart/Middle Note', 'link', 'Scents/Notes', 'Size', 'Top Note','price']]

In [7]:
# Drop null values if the record have ('Base Note','Heart/Middle Note','Top Note') all null values 
noon=noon.dropna(subset=['Base Note','Heart/Middle Note','Top Note'], how='all')

In [8]:
# Handling data inconsistency 
noon['Heart/Middle Note'] = noon['Heart/Middle Note'].str.replace(r' \bAnd\b',',')
noon['Heart/Middle Note'] = noon['Heart/Middle Note'].str.replace(r' \band\b',',')
noon['Heart/Middle Note'] = noon['Heart/Middle Note'].str.replace(' &',',')
noon['Base Note'] = noon['Base Note'].str.replace(r' \bAnd\b',',')
noon['Base Note'] = noon['Base Note'].str.replace(r' \band\b',',')
noon['Base Note'] = noon['Base Note'].str.replace(' &',',')
noon['Scents/Notes'] = noon['Scents/Notes'].str.replace(r' \bAnd\b',',')
noon['Scents/Notes'] = noon['Scents/Notes'].str.replace(r' \band\b',',')
noon['Scents/Notes'] = noon['Scents/Notes'].str.replace(' &',',')
noon['Top Note'] = noon['Top Note'].str.replace(r' \bAnd\b',',')
noon['Top Note'] = noon['Top Note'].str.replace(r' \band\b',',')
noon['Top Note'] = noon['Top Note'].str.replace(' &',',')
noon['Department'] = noon['Department'].str.strip()

In [9]:
# TOP NOTE DUPES FFILL

# This data set will include only the duplicate values
dupes=noon[noon.duplicated(['name','Department','Size'],keep=False)]

# Sort the dataset (null Top Note values will be at the bottom)
dupes = dupes.sort_values(by=['name', 'Top Note'])

names_col = dupes['name']

# Fill null value using forward filling method and group by the name 
dupes['Top Note']=dupes.groupby(['name'])['Top Note'].ffill()

dupes['name']=names_col


# BASE NOTE DUPES FFILL

# Sort the dataset (null Base Note values will be at the bottom)
dupes = dupes.sort_values(by=['name', 'Base Note'])

names_col = dupes['name']

# Fill null value using forward filling method and group by the name 
dupes['Base Note']=dupes.groupby(['name'])['Base Note'].ffill()
dupes['name']=names_col


# HEARF/MIDDLE NOTE DUPES FFILL

# Sort the dataset (null Heart/Middle Note values will be at the bottom)
dupes = dupes.sort_values(by=['name', 'Heart/Middle Note'])

names_col = dupes['name']

# Fill null value using forward filling method and group by the name 
dupes['Heart/Middle Note']=dupes.groupby(['name'])['Heart/Middle Note'].ffill()
dupes['name']=names_col


# SCENTS/NOTES DUPES FFILL

# Sort the dataset (null Scents/Notes values will be at the bottom)
dupes = dupes.sort_values(by=['name', 'Scents/Notes'])

names_col = dupes['name']

# Fill null value using forward filling method and group by the name 
dupes['Scents/Notes']=dupes.groupby(['name'])['Scents/Notes'].ffill()
dupes['name']=names_col

In [10]:
# Sort the dataset 
noon_sort = noon.sort_values(by=['name', 'Top Note'])

# Drop all duplicate values from the original dataset 
noon_sort=noon_sort.drop_duplicates(subset=['name','Department'],keep=False)


# Drop duplicate values from the dupes dataset and keep only one record  
dupes=dupes.drop_duplicates(subset=['name'])

# Add duplicate recordes back to the original dataset
noon_sort=noon_sort.append(dupes)

# Sort by name 
noon_sort = noon_sort.sort_values(by=['name'])

# View dataset after handling duplicate values
print(f"Rows: {noon_sort.shape[0]}\nColumns: {noon_sort.shape[1]}")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(noon_sort.head(2))


Rows: 5192
Columns: 9


,name,Base Note,Department,Heart/Middle Note,link,Scents/Notes,Size,Top Note,price
4408,24 Azur EDP 100ml,"Tonka, Musk, Dry Wood, Vanilla",Unisex,"Artemisia Cedarwood, Neroli",/uae-en/azur-edp-100ml/N38430201A/p?o=e5afe89a...,Woody,100 ml,"Bergamot, Juniper Berry, Mara Starwaberry",115.00
4967,24 Elixir Gold EDP 100ml,"Cedarwood, Amber, Gaiac Wood, Vanilla",Unisex,"Coco, Rose, Honey, Safran",/uae-en/elixir-gold-edp-100ml/N38706495A/p?o=b...,Spicy,100 ml,"Cardamom, Pine",99.65


In [11]:
# Make copy of the data set 
n=noon_sort.copy()

# Keep the first duplicate
n = n[~n.index.duplicated(keep='first')]

In [12]:
# Fixing the links column  
n['link'] = 'https://www.noon.com' + n['link']

In [13]:
# Handling data inconsistency in name column 

n['name'] = n['name'].str.replace("Eau De Perfume", "EDP", case=False)
n['name'] = n['name'].str.replace("Eau de Parfum", "EDP", case=False)
n['name'] = n['name'].str.replace("Eau De Pafum", "EDP", case=False)
n['name'] = n['name'].str.replace("Eau De Pefum", "EDP", case=False)
n['name'] = n['name'].str.replace("Eau De Pefume", "EDP", case=False)
n['name'] = n['name'].str.replace("Eau De Perfum", "EDP", case=False)
n['name'] = n['name'].str.replace("De Parfum", "EDP", case=False)
n['name'] = n['name'].str.replace(r"\bDPerfume\b", "EDP", case=False)
n['name'] = n['name'].str.replace(r"\bedp\b", "EDP", case=False)

n['name'] = n['name'].str.replace(r"\bPerfum\b", "Parfum", case=False)
n['name'] = n['name'].str.replace(r"\bPafum\b", "Parfum", case=False)

n['name'] = n['name'].str.replace(r"\bColonge\b", "EDC", case=False)


In [14]:
# Fill in the null values inside the size column based on the written size in the name column 
n["Size"]= n["Size"].str.strip()
n["Sizes"]=n["name"].str.split().str[-1]
n["Size"]=n["Size"].fillna(n["Sizes"])
n.drop(['Sizes'], axis=1,inplace=True)


In [15]:
# Handling data inconsistency in name column 

n['name']= n['name'].str.replace(r'\d+ \bMililitre\b', '' , case=False)
n['name']= n['name'].str.replace(r'\d+\bMililitre\b', '', case=False) 
n['name']= n['name'].str.replace(r'\d+ \bmillimeter\b', '' , case=False)   
n['name']= n['name'].str.replace(r'\d+\bmillimeter\b', '', case=False) 
n['name']= n['name'].str.replace(r'\d+ \bml\b', '', case=False)
n['name']= n['name'].str.replace(r'\d+\bml\b', '', case=False)
n['name']= n['name'].str.replace(r'\d+ounce', '', case=False) 
n['name']= n['name'].str.replace(r'\d+ \bounce\b', '', case=False) 
n['name']= n['name'].str.replace(r'\d+ \bx\b', '', case=False) 
n['name']= n['name'].str.replace(r'\d+\bcm\b', '', case=False)
n['name']= n['name'].str.replace(r'\b\d+.\b', '', case=False)
n['name']= n['name'].str.replace(r'\bMillilitre\b', '', case=False)  
n['name']= n['name'].str.replace(r'\bml\b', '', case=False)
n['name']= n['name'].str.replace(r'\billileter\b', '', case=False)
n['name']= n['name'].str.replace(r'\bililiter\b', '', case=False)
n['name']= n['name'].str.replace(r'\billilitre\b', '', case=False)



In [16]:
# Function copy the selected word from a string 
def find_word(word,string):
    search_string=string
    word=word
    lword=len(word)
    start_index=search_string.find(word)
    extracted_string= search_string[start_index:start_index+lword]
    return extracted_string
    
# Copy the concentration type from the name column
n['EDP'] = n['name'].apply(lambda x: find_word('EDP',x))
n['EDC'] = n['name'].apply(lambda x: find_word('EDC',x))
n['EDT'] = n['name'].apply(lambda x: find_word('EDT',x))
n['Parfum'] = n['name'].apply(lambda x: find_word('Parfum',x))

# Combine into a single column
n['concentration']=n['EDP']+n['EDC']+n['EDT']+n['Parfum']

# Handling data inconsistency and null values in concentration column
n['concentration'] = n['concentration'].str.strip()
n['concentration'] = n['concentration'].str.replace(r"\bEDPParfum\b", "EDP", case=False)
n['concentration'] = n['concentration'].str.replace(r"\bEDPEDT\b", "EDT", case=False)
n['concentration'] = n['concentration'].str.replace(r"\bEDTParfum\b", "EDT", case=False)
n['concentration']=n['concentration'].replace(r'', np.NaN)

# Fill null values with the mode 
n['concentration'] = n['concentration'].fillna("EDP")


In [17]:
# Handling data inconsistency in name column 

n['name'] = n['name'].str.replace(r"\bEDP\b", "", case=False)
n['name'] = n['name'].str.replace(r"\bParfum\b", "", case=False)
n['name'] = n['name'].str.replace(r"\bEDC\b", "", case=False)
n['name'] = n['name'].str.replace(r"\bEDT\b", "", case=False)
n['name'] = n['name'].str.replace("&amp;", "&", case=False)
n['name'] = n['name'].str.replace(r" \be\b", "")
n['name']= n['name'].str.replace('\d+ml', '', case=False)
n['name'] = n['name'].str.replace("  ", "", case=False)
n['name'] = n['name'].str.replace("   ", "", case=False)
n.fillna('',inplace=True)

In [18]:
# Drop unnecessarily columns 
n.drop(['EDP','EDC','EDT','Parfum'], axis=1,inplace=True)
noon_clean = n.copy()

In [19]:
# to lowercase
noon_clean['Size'] = noon_clean['Size'].str.lower()
noon_clean['Top Note'] = noon_clean['Top Note'].str.lower()  
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.lower()
noon_clean['Base Note'] = noon_clean['Base Note'].str.lower()

In [20]:
# Handling data inconsistency in Size column 
noon_clean['Size'] = noon_clean['Size'].str.replace(" ", "")
noon_clean['Size'] = noon_clean['Size'].str.replace("mililitre", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("millimeter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mililiter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("millileter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("millilitre", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mililiter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mllimeter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mliliter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mllileter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mllilitre", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mllimeter", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace(r"1xedp80,1xedp", "")
noon_clean['Size'] = noon_clean['Size'].str.replace(r"3.5x2.4x9.4cm", "" , regex =False)
noon_clean['Size'] = noon_clean['Size'].str.replace(r"edp125,bodylotion100,parfumextract", "")
noon_clean['Size'] = noon_clean['Size'].str.replace(r"(edp100,edp75)", "75" , regex =False)
noon_clean['Size'] = noon_clean['Size'].str.replace(r"59x59x110cm", "" , regex =False)
noon_clean['Size'] = noon_clean['Size'].str.replace(r"2xedenedp(50)ml", "2x50ml" , regex =False)
noon_clean['Size'] = noon_clean['Size'].str.replace(r"\bg\b", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("extrememenedp100,extremewomenedp", "")
noon_clean['Size'] = noon_clean['Size'].str.replace("4.3x2.3x7.9cm", "" , regex =False)
noon_clean['Size'] = noon_clean['Size'].str.replace("mil", "ml")
noon_clean['Size'] = noon_clean['Size'].str.replace("mlml", "ml")



In [21]:
# Remove Size column values that have only 'ml' without a number 
for x in noon_clean.Size.index:
    if noon_clean.Size.loc[x] == 'ml':
        noon_clean.Size[x]= ''


<ipython-input-21-fdd5bdebaafb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noon_clean.Size[x]= ''


In [22]:
# Handling data inconsistency in name column 
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmililitre\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmil\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmillimeter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmililiter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmillileter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmillilitre\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmililiter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmlml\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmllimeter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmliliter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmllileter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmllilitre\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bmllimeter\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b1xedp80,1xedp\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b3.5x2.4x9.4cm\b", "" , regex =False, case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bedp125,bodylotion100,parfumextract\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b(edp100,edp75)\b", "" , regex =False, case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b59x59x110cm\b", "" , regex =False, case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b2xedenedp(50)ml\b", "" , regex =False, case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bg\b", "ml", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bextrememenedp100,extremewomenedp\b", "", case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\b4.3x2.3x7.9cm\b", "" , regex =False, case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\d+millimeter", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\d+mililiter", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\d+Mililitre", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\d+ounce", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\bml\b", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r" \bm\b", " ",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\billilitre\b", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\d+.", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(",", "",case=False)
noon_clean['name'] = noon_clean['name'].str.replace("For Women", " For Women ",case=False)
noon_clean['name'] = noon_clean['name'].str.replace("For Men", " For Men ",case=False)
noon_clean['name'] = noon_clean['name'].str.replace("w ood", "wood",case=False)
noon_clean['name'] = noon_clean['name'].str.replace(r"\s-", "",case=False)
n['name'] = n['name'].str.replace("  ", " ", case=False)
n['name'] = n['name'].str.replace("   ", " ", case=False)

In [23]:
# Function to remove whitespace from the beginning and end of a string 
def remove_first_end_spaces(string):
    return "".join(string.rstrip().lstrip())

# Run the function for all columns 
for col in n.columns: 
    if(col!='price'):
        noon_clean[col] = noon_clean[col].apply(lambda x: remove_first_end_spaces(x))

In [24]:
# Export the dataset file 
noon_clean.to_csv('./data/noon_clean_EDA.csv', index = False, header=True)

## <span style="color:#55BB99"> 2- Model Cleanup</span>
In this section of the notebook we do a specific cleanup for the whole dataset in preparation for the model. Because the notes have to be prepared in a different way than for the EDA specifically for the wordclouds and the heatmaps. 

In [25]:
# some notes like 'gaiac wood' were spelled with a 'ï', to unify them I wrote this code
noon_clean['Base Note']=noon_clean['Base Note'].str.replace('ï', 'i')
noon_clean['Top Note']=noon_clean['Top Note'].str.replace('ï', 'i')
noon_clean['Heart/Middle Note']=noon_clean['Heart/Middle Note'].str.replace('ï', 'i')

### <span style="color:#726060"> 2.1- Base Note Cleanup</span>

In [26]:
# cleaning the weird symbols 
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(r'[\(\)\.\،]', "")
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(r'[\/\-\:\;]', ",")
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(r'\bor\b', "," ,case=False)
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(r"\b\xa0\b","")

In [27]:
# this method takes in a string and a list of stop words and removes any of the stop words found in the string
def normalize_word(word,stopwords):
    stopwords = stopwords
    querywords = word.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result

In [28]:
# in order for the stopword function to work properly there needs to be spaces between the commas
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(',', " , " )

In [29]:
# defining base note stop words
stopwords_basenote=['si','code', 'profumo', 'bean','beans','oil','leaves', 'of','precious' ,'blended', 
                    'with','accord','a', 'combination', 'perfume', 'perfumes','twin',
                     'base','etetee','md','al','woman' ,'skin','are','valley','leave', 
                     'steady','on','the' ,'like' ,'scent','warming','shades','kick','fragrance',
                    'ending','consists','s','by', 'banana' ,'republic','100%' ,'new',
                   'this', 'item', 'cannot', 'be' ,'exchanged' ,'or', 'returned']

In [30]:
# applying the normalization function to the base note column
noon_clean['Base Note'] = noon_clean['Base Note'].apply(lambda x: normalize_word(x,stopwords_basenote))

In [31]:
# removing all white spaces to unify the notes
noon_clean['Base Note'] = noon_clean['Base Note'].str.replace(" ","")

### <span style="color:#726060"> 2.2- Top Note Cleanup</span>

In [32]:
# cleaning the weird symbols 
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(r'[\(\)\.\:\-]', '')

# list of words to replace with ','
replace_t = ["mixed with", 'along with', 'mingle with', r'\bwith\b',r'\bor\b','heart note','base note',';' ]

# replace 
for x in replace_t: 
    noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(x , ',' , regex = True)

# more cleaning
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(r'\blemone\b' , 'lemon')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('dew drop ' , 'dewdrop')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('ylan-ylan' , 'ylang ylang' , regex = False )
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('bouquet of roses' , 'rose')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('\xa0red fruits ' , 'red fruits', regex = False)
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('\xa0plum' , 'plum', regex = False)
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('lily-of-the-valley' , 'lily', regex = False)
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace('sicilian-lemon' , 'sicilian lemon', regex = False)
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(r'\bwooden\b' , 'woody ')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(', ,' , ',')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(r'(?=[^\s])([,])' , ' ,')
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(r'([,])(?=[^\s])' , ', ')

In [33]:
# defining top note stop words
stopwords_topnote=['code', 'profumo', 'bean','beans','oil','leaves', 'm','exciting' ,'powerful', 
                    'bright','big ','je', 'accord', 'perfume', 'perfumes','of',
                     'creamy','etetee','yummy ','exotic','new' ,'note','cut','hint','rich', 
                     'tone',  'music' ,'y' ,'a' ,'men','an','essence','er','fragrances',
                    'hint','the' ,'valley', 'as', 'well','flavors', 'heart', 'at', 'extreme', 'women', 
                   'edp', 'fragrance', 'for','sprinkled', 'elegant' , 'dazzling', 'wave', 'juiciness', 'cocktail',
                   'aromatic', 'open', 'scent', 'inviting', 'features' , 'notes', 'that', 'wet', 'begin', 'freshly', 'plucked', 
                   'iso', 'e',  'super', 'st', 'c', 'w' , 'combine', 'playful'
                  ]
# applying the normalization function to the top note column
noon_clean['Top Note'] = noon_clean['Top Note'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_topnote)]))

In [34]:
# removing all white spaces to unify the notes
noon_clean['Top Note'] = noon_clean['Top Note'].str.replace(" ","")

### <span style="color:#726060"> 2.3- Middle Note Cleanup</span>

In [35]:
# cleaning the weird symbols 
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(r'[\(\)\.\:\-]', '')

# list of words to replace with ','
replace_m = [r'\bwith\b',r'\bor\b',';' ]

# replace 
for x in replace_m: 
    noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(x , ',' , regex = True)

# more cleaning
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('\xa0' , ' ')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('lilyofthevalley' , 'lily')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('ylangylan' , 'ylang ylang' , regex = False )
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('ylangylang' , 'ylang ylang' , regex = False )
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('de mai' , '')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('lilyofthevalley' , 'lily', regex = False)
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('lilyofthe valley' , 'lily', regex = False)
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('lillyofthevalley' , 'lily', regex = False)
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace('mint orange apple' , 'mint, orange, apple', regex = False)
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(', ,' , ',')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(',,' , ',')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(r'(?=[^\s])([,])' , ' ,')
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(r'([,])(?=[^\s])' , ', ')

In [36]:
# defining middel note stop words
stopwords_middelnote=['code', 'profumo','oil','leaves', 'other','exciting' ,'powerful', 
                    'bright','big ','je', 'accord', 'perfume', 'perfumes','tm',
                     'creamy','from','bouqet ','dewy','new' ,'note','cut','hint','rich', 
                     '2',  'middle' ,'1' ,'twin' ,'aroma','layer','accord','delicate','whipped',
                    'nuance','alliance' ,'in', 'holy', 'perfume','base', 'then', 'city', 'center', 'mixed', 
                   'edp', 'fragrance', 'an','harmonies', 'perfume' , 'isis', 'precious', 'temptation', 'intoxicating',
                   'aromatic', 'absolute', 'lady', 'needles', 'notes' , 'delicate', 'playful', 'entangle', 'seduction', 'king', 'plucked', 
                   'living', 'dried',  'touch', 'valley', 'the', 'of' , 'secure', 'shopping', 'container', 'shape', 'is', 
                    'beautiful', 'with', 'a', 'nice', 'gift', 'box', 'melded', 'with', 'earthy', 'hints', 'for',
                      'by', 'banana', 'republic', 'desert' , 'depened' , 'crisp' , 'candied' , 'needles' , 'creeper' , 'salted' , 
                      'tincture' , 'verbena' , 'serene' , 'essential' , 'rich' , 'fused' , 'pretty' , 'bouquet' , 
                  ]

# applying the normalization function to the middel note column
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_middelnote)]))

In [37]:
# removing all white spaces to unify the notes
noon_clean['Heart/Middle Note'] = noon_clean['Heart/Middle Note'].str.replace(" ","")

In [39]:
# Export the dataset file 
noon_clean.to_csv('./data/noon_clean_Model.csv', index = False, header=True)